In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import time

import fannypack
from lib import dpf, panda_models, panda_datasets, panda_training

print(torch.__version__, np.__version__)

1.4.0 1.18.1


In [3]:
# Experiment configuration
experiment_name = "dpf_all_sensors_tbtt5"
dataset_args = {
    'use_proprioception': True,
    'use_haptics': True,
    'use_vision': True,
    'vision_interval': 2,
}

In [10]:
# Create models & training buddy

pf_model = panda_models.PandaParticleFilterNetwork()

buddy = fannypack.utils.Buddy(
    experiment_name,
    pf_model,
    optimizer_names=["e2e", "dynamics", "measurement"]
)

[buddy-dpf_all_sensors_tbtt5] Using device: cuda
[buddy-dpf_all_sensors_tbtt5] No checkpoint found


In [11]:
print(next(pf_model.parameters()).device)

cuda:0


# Dynamics Model Pre-Training

In [43]:
# dynamics_trainset = panda_datasets.PandaDynamicsDataset(
#     "data/pull-test.hdf5",
#     "data/push-test.hdf5",
#     **dataset_args
# )

In [22]:
# dataloader = torch.utils.data.DataLoader(dynamics_trainset, batch_size=256, shuffle=True, num_workers=2)

# for i in range(0):
#     print("Training epoch", i)
#     panda_training.train_dynamics(buddy, pf_model, dataloader, log_interval=10)
#     print()


# Measurement Model Pre-Training

In [44]:

measurement_trainset = panda_datasets.PandaMeasurementDataset(
    "data/pushset_medium.hdf5",
    samples_per_pair=10,
    **dataset_args
)

Widget Javascript not detected.  It may not be installed or enabled properly.


Loaded 40000 points


In [34]:
measurement_trainset_loader = torch.utils.data.DataLoader(measurement_trainset, batch_size=64, shuffle=True, num_workers=16)
for i in range(15):
    print("Training epoch", i)
    panda_training.train_measurement(buddy, pf_model, measurement_trainset_loader, log_interval=20)
    print()

# measurement_trainset_loader = torch.utils.data.DataLoader(measurement_trainset, batch_size=64, shuffle=True, num_workers=2)
# for i in range(1):
#     print("Training epoch", i)
#     panda_training.train_measurement(buddy, pf_model, measurement_trainset_loader, log_interval=100)
#     print()

# measurement_trainset_loader = torch.utils.data.DataLoader(measurement_trainset, batch_size=256, shuffle=True, num_workers=2)
# for i in range(1):
#     print("Training epoch", i)
#     panda_training.train_measurement(buddy, pf_model, measurement_trainset_loader, log_interval=100)
#     print()



Training epoch 0


Widget Javascript not detected.  It may not be installed or enabled properly.



Epoch loss: 7639.6685

Training epoch 1


Widget Javascript not detected.  It may not be installed or enabled properly.


IndexError: list index out of range

In [ ]:
buddy.save_checkpoint(label="before_e2e_training")

# End-to-end Training

In [6]:
# Create end-to-end dataset

e2e_trainset = panda_datasets.PandaParticleFilterDataset(
    "data/gentle_push_10.hdf5",
    subsequence_length=4,
    particle_count=30,
    particle_variances=(.0005, .0005),
    **dataset_args
)

Parsed data: 60 active, 0 inactive
Keeping (inactive): 0


In [9]:
##### Train end-to-end
pf_model.dynamics_model.state_noise = (0.005, 0.005)

pf_model.freeze_measurement_model = False
pf_model.freeze_dynamics_model = True

# e2e_trainset_loader = torch.utils.data.DataLoader(e2e_trainset, batch_size=32, shuffle=True, num_workers=2)
# for i in range(3):
#     print("Training epoch", i)
#     panda_training.train_e2e(buddy, pf_model, e2e_trainset_loader, loss_type="mse", log_interval=100)

e2e_trainset_loader = torch.utils.data.DataLoader(e2e_trainset, batch_size=64, shuffle=True, num_workers=2)

for i in enumerate(e2e_trainset_loader):
    print(i)

for i in range(20):
    print("Training epoch", i)
    panda_training.train_e2e(buddy, pf_model, e2e_trainset_loader, loss_type="mse")

(0, [tensor([[[-0.3254,  0.9638],
         [-0.3318,  1.0160],
         [-0.3385,  0.9713],
         ...,
         [-0.3437,  0.9915],
         [-0.3772,  1.0168],
         [-0.3394,  0.9871]],

        [[-0.3263,  0.9762],
         [-0.3651,  1.0144],
         [-0.3363,  0.9408],
         ...,
         [-0.3381,  0.9801],
         [-0.3253,  0.9780],
         [-0.3602,  1.0210]],

        [[-0.3739,  0.9961],
         [-0.3145,  0.9507],
         [-0.3255,  0.9497],
         ...,
         [-0.3602,  0.9768],
         [-0.3524,  0.9734],
         [-0.3548,  0.9613]],

        ...,

        [[-0.3834,  1.0012],
         [-0.3584,  0.9910],
         [-0.3608,  0.9937],
         ...,
         [-0.3512,  0.9943],
         [-0.3724,  0.9857],
         [-0.3298,  1.0359]],

        [[-0.3078,  0.9271],
         [-0.3442,  0.9426],
         [-0.3185,  0.9418],
         ...,
         [-0.3398,  0.9275],
         [-0.3591,  0.9676],
         [-0.3400,  0.9507]],

        [[-0.3433,  0.9913],
  

AssertionError: 

In [ ]:
buddy.save_checkpoint()

# Model eval

In [35]:
eval_trajectories = panda_datasets.load_trajectories(
    "data/pushset_small.hdf5",
    **dataset_args
)

### Final Model

In [37]:
pred, actual = panda_training.rollout(pf_model, eval_trajectories, start_time=0, max_timesteps=1000, particle_count=200, noisy_dynamics=True)
panda_training.eval_rollout(pred, actual, plot=True)
print(pf_model.dynamics_model.state_noise)

Widget Javascript not detected.  It may not be installed or enabled properly.


RuntimeError: invalid multinomial distribution (encountering probability entry < 0)

### Model without end-to-end training

In [ ]:
pf_model.dynamics_model.state_noise = (0.005, 0.005)


# Back up model
buddy.save_checkpoint()

# Load the pre-end-to-end-training version of the model & evaluate
buddy.load_checkpoint(label="before_e2e_training")

pred, actual = panda_training.rollout(pf_model, eval_trajectories, start_time=0, max_timesteps=1000, particle_count=200, noisy_dynamics=True)
panda_training.eval_rollout(pred, actual, plot=True)

# Restore model
buddy.load_checkpoint()